# ۲ – ساخت RAG ساده


In [2]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence_transformers-5.2.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
Using cached sentence_transformers-5.2.0-py3-none-any.whl (493 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------------------- -------------------- 1.3/2.7 MB 4.2 MB/s eta 0:00:01
   -------------------------------------- - 2.6/2.7 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 5.1 MB/s eta 0:00:00
Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl (341 kB)
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.9 MB 2.7 MB/s eta 0:00:07
   -- ------------------------------------- 1.3/18.9 MB 3.7 MB/s eta 0:00:05
   ----- ---------------------------------- 2.6/18.9 MB 4.4 MB/s eta 0:00:04
   ----------- ---------------------------- 5.5/18.9 MB 6.8 MB/s eta 0:00:02
   ---------------- ----------------------- 7.6/18.9 MB 7.5 MB/s eta 0:00:02
   --------------------- ------------------ 10.0/18.9 MB 8.2 MB/s eta 0:00:02
   -------------------------- ------------- 12.6/18.9 MB 8.8 MB/s eta 0:00:01
   -------------------------------- ------- 15.2/18.9 MB 9.3 MB/s eta 0:00:01
   ------------------------------------- -- 17.8/18.9 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 18.9/18.9 MB 9.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import faiss
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('../data/processed/news_subset.csv')
df['full_text'] = df['title'] + ": " + df['text']
df.head(2)

,id,title,short_link,service,subgroup,abstract,text,tags,published_datetime,agency_name,full_text
0,149198,رئیس اتحادیه ناشران و کتابفروشان تهران: ارز 42...,http://fna.ir/4b6no,فرهنگ,کتاب و ادبیات,رئیس اتحادیه ناشران و کتابفروشان تهران گفت: ار...,به گزارش خبرنگار کتاب و ادبیات خبرگزاری فارس، ...,"هومان حسن پور, رئیس اتحادیه ناشران و کتاب فروش...",2021-10-16 01:20:24,FarsNews,رئیس اتحادیه ناشران و کتابفروشان تهران: ارز 42...
1,61973,وزنه‌برداری قهرمانی آسیا| داودی مدال طلای یکضر...,http://fna.ir/17yaj,ورزشی,کشتی و وزنه برداری,علی داودی در حرکت یکضرب دسته 109+ کیلوگرم رقاب...,به گزارش خبرنگار ورزشی خبرگزاری فارس، امروز (ی...,"قهرمانی آسیا, مدال طلا, وزنه‌برداری, علی داودی",2021-04-25 03:41:57,FarsNews,وزنه‌برداری قهرمانی آسیا| داودی مدال طلای یکضر...


In [6]:
def simple_chunker(text, chunk_size=800, overlap=150):
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + chunk_size, text_len)
        chunks.append(text[start:end])
        start += (chunk_size - overlap)
    return chunks

chunk_data = []
for idx, row in df.iterrows():
    chunks = simple_chunker(row['full_text'])
    for c_idx, chunk in enumerate(chunks):
        chunk_data.append({
            'chunk_id': f"{idx}_{c_idx}",
            'doc_id': idx,
            'text': chunk,
            'category': row.get('category', 'unknown'),
            'date': row.get('date', '')
        })

df_chunks = pd.DataFrame(chunk_data)
print(f"Total chunks: {len(df_chunks)}")

Total chunks: 36272


In [7]:
semantic_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
semantic_embeddings = semantic_model.encode(df_chunks['text'].tolist(), show_progress_bar=True)

tfidf = TfidfVectorizer(max_features=384)
tfidf_matrix = tfidf.fit_transform(df_chunks['text'])
lexical_embeddings = tfidf_matrix.toarray().astype('float32')

combined_embeddings = np.hstack([semantic_embeddings, lexical_embeddings])
combined_embeddings = combined_embeddings.astype('float32')

print(f"Semantic shape: {semantic_embeddings.shape}")
print(f"Lexical shape: {lexical_embeddings.shape}")
print(f"Combined shape: {combined_embeddings.shape}")

Batches:   0%|          | 0/1134 [00:00<?, ?it/s]

Semantic shape: (36272, 384)
Lexical shape: (36272, 384)
Combined shape: (36272, 768)


In [8]:
d_semantic = semantic_embeddings.shape[1]
index_semantic = faiss.IndexFlatIP(d_semantic)
index_semantic.add(semantic_embeddings)

d_combined = combined_embeddings.shape[1]
index_combined = faiss.IndexFlatIP(d_combined)
index_combined.add(combined_embeddings)

In [9]:
def retrieve(question, top_k=3, mode='semantic'):
    if mode == 'semantic':
        q_emb = semantic_model.encode([question])
        D, I = index_semantic.search(q_emb, top_k)
    elif mode == 'combined':
        s_emb = semantic_model.encode([question])
        l_emb = tfidf.transform([question]).toarray().astype('float32')
        q_emb = np.hstack([s_emb, l_emb])
        D, I = index_combined.search(q_emb, top_k)
    
    results = []
    for idx in I[0]:
        if idx < len(df_chunks):
            results.append(df_chunks.iloc[idx]['text'])
    return results

In [10]:
def answer(question, mode='semantic'):
    retrieved_texts = retrieve(question, top_k=3, mode=mode)
    context = "\n---\n".join(retrieved_texts)
    final_answer = f"سوال: {question}\n\nمستندات یافت شده:\n{context}\n\n(پاسخ نهایی بر اساس ترکیب مستندات بالا تولید می‌شود)"
    return final_answer

In [11]:
q_sample = "نرخ تورم چقدر است؟"
print(answer(q_sample, mode='combined'))

سوال: نرخ تورم چقدر است؟

مستندات یافت شده:
ا ثبت کرده است. این افزایش نشان می‌دهد، در بحبوحه بحران زنجیره تامین در جهان، نرخ بالای تورم همچنان در این کشور تا سال آینده ادامه خواهد یافت. وزارت کار آمریکا اعلام کرد شاخص قیمت مصرف کننده در ماه گذشته میلادی 0.9 درصد افزایش یافت و در ماه سپتامبر نیز رشد 0.4 درصدی را ثبت کرده بود. در 12 ماه منتهی به اکتبر، شاخص قیمت مصرف کننده 6.2 درصد رشد داشته است. این رقم، بیشترین میزان سالانه از نوامبر 1990 به شمار می‌رود. تورم در آمریکا با فشار اقتصادی ناشی از موج تابستانی کرونا و انتشار سویه دلتا و بحران‌های عرضه دوباره شدت گرفته است. افزایش تورم در کشورها از جمله آمریکا به معضلی بزرگ تبدیل شده و نارضایتی بین مردم ایجاد کرده است، فدرال رزرو آمریکا افزایش تورم را به دلیل بالا رفتن قیمت سوخت می‌داند. بانک جهانی هم گزارش داده است: رشد قیمت کالاهای اساسی به تورم کشورها انجامیده و پیش بینی کرده که در سا
---
نرخ تورم مصر در حال نزدیک شدن به بالاترین سطح تاریخی: به گزارش خبرگزاری مهر به نقل از روسیا الیوم، بر اساس داده‌های مرکز آمار مصر، نرخ تورم سالانه در ش